In [1]:
### インポート ###

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from datetime import datetime
import time
from selenium.webdriver.common.keys import Keys
import re
import csv
import pandas as pd
from datetime import datetime, timedelta
from dataclasses import dataclass, field
from typing import List, TypeVar, Generic

In [18]:
### 定数 ###

# 設定ファイル読み込み
df_config_racecourse = pd.read_excel("C:\\Users\\ryo\\Downloads\\config.xlsx", sheet_name="racecourse", header=0)
df_config_style = pd.read_excel("C:\\Users\\ryo\\Downloads\\config.xlsx", sheet_name="style", header=0)
df_config_scrape = pd.read_excel("C:\\Users\\ryo\\Downloads\\config.xlsx", sheet_name="scrape", header=None, index_col=0)
df_config_netkeiba = pd.read_excel("C:\\Users\\ryo\\Downloads\\config.xlsx", sheet_name="netkeiba", header=None, index_col=0)
# 対象競馬場とレース
PLACE_MAP = df_config_racecourse.set_index('key')['value'].to_dict()
# 脚質
STYLE_MAP = df_config_style.set_index('key')['value'].to_dict()
# レース番号
RACE_NUMBERS = [f"{i:02d}" for i in range(1, 13)]
# スクレイピング
PATH_CHROME_DRIVER = df_config_scrape.loc["PATH_CHROME_DRIVER"].iloc[0]
# netkeiba
LOGIN_URL = df_config_netkeiba.loc["LOGIN_URL"].iloc[0]
LOGIN_ID = df_config_netkeiba.loc["LOGIN_ID"].iloc[0]
LOGIN_PASSWORD = df_config_netkeiba.loc["LOGIN_PASSWORD"].iloc[0]
RACE_LIST_URL = df_config_netkeiba.loc["RACE_LIST_URL"].iloc[0]
MODE = df_config_netkeiba.loc["MODE"].iloc[0]
# レース単位
if MODE == "shutuba":
    SELECTOR_RACECOURSE = df_config_netkeiba.loc["SELECTOR_RACECOURSE_SHUTUBA"].iloc[0]
    SELECTOR_RACE_NUMBER = df_config_netkeiba.loc["SELECTOR_RACE_NUMBER_SHUTUBA"].iloc[0]
    SELECTOR_NUM_HORSES = df_config_netkeiba.loc["SELECTOR_NUM_HORSES_SHUTUBA"].iloc[0]
    XPATH_RACE_INFO = df_config_netkeiba.loc["XPATH_RACE_INFO_SHUTUBA"].iloc[0]
elif MODE == "result":
    SELECTOR_RACECOURSE = df_config_netkeiba.loc["SELECTOR_RACECOURSE_RESULT"].iloc[0]
    SELECTOR_RACE_NUMBER = df_config_netkeiba.loc["SELECTOR_RACE_NUMBER_RESULT"].iloc[0]
    SELECTOR_NUM_HORSES = df_config_netkeiba.loc["SELECTOR_NUM_HORSES_RESULT"].iloc[0]
    XPATH_RACE_INFO = df_config_netkeiba.loc["XPATH_RACE_INFO_RESULT"].iloc[0]
XPATH_RELIABILITY = df_config_netkeiba.loc["XPATH_RELIABILITY"].iloc[0]
XPATH_OPINION = df_config_netkeiba.loc["XPATH_OPINION"].iloc[0]
XPATH_CORNER_1 = df_config_netkeiba.loc["XPATH_CORNER_1"].iloc[0]
XPATH_CORNER_2 = df_config_netkeiba.loc["XPATH_CORNER_2"].iloc[0]
XPATH_CORNER_3 = df_config_netkeiba.loc["XPATH_CORNER_3"].iloc[0]
XPATH_RANK_1_CORNER = df_config_netkeiba.loc["XPATH_RANK_1_CORNER"].iloc[0]
XPATH_RANK_2_CORNER = df_config_netkeiba.loc["XPATH_RANK_2_CORNER"].iloc[0]
XPATH_RANK_3_CORNER = df_config_netkeiba.loc["XPATH_RANK_3_CORNER"].iloc[0]
XPATH_RANK_4_CORNER = df_config_netkeiba.loc["XPATH_RANK_4_CORNER"].iloc[0]
# 馬単位（shutuba）
XPATH_HORSE_NUMBER_1_SHUTUBA = df_config_netkeiba.loc["XPATH_HORSE_NUMBER_1_SHUTUBA"].iloc[0]
XPATH_HORSE_NUMBER_2_SHUTUBA = df_config_netkeiba.loc["XPATH_HORSE_NUMBER_2_SHUTUBA"].iloc[0]
XPATH_HORSE_LINK_1_1_SHUTUBA = df_config_netkeiba.loc["XPATH_HORSE_LINK_1_1_SHUTUBA"].iloc[0]
XPATH_HORSE_LINK_1_2_SHUTUBA = df_config_netkeiba.loc["XPATH_HORSE_LINK_1_2_SHUTUBA"].iloc[0]
XPATH_HORSE_LINK_2_1_SHUTUBA = df_config_netkeiba.loc["XPATH_HORSE_LINK_2_1_SHUTUBA"].iloc[0]
XPATH_HORSE_LINK_2_2_SHUTUBA = df_config_netkeiba.loc["XPATH_HORSE_LINK_2_2_SHUTUBA"].iloc[0]
XPATH_JOCKEY_LINK_1_SHUTUBA = df_config_netkeiba.loc["XPATH_JOCKEY_LINK_1_SHUTUBA"].iloc[0]
XPATH_JOCKEY_LINK_2_SHUTUBA = df_config_netkeiba.loc["XPATH_JOCKEY_LINK_2_SHUTUBA"].iloc[0]
XPATH_POPULARITY_1_SHUTUBA = df_config_netkeiba.loc["XPATH_POPULARITY_1_SHUTUBA"].iloc[0]
XPATH_POPULARITY_2_SHUTUBA = df_config_netkeiba.loc["XPATH_POPULARITY_2_SHUTUBA"].iloc[0]
XPATH_ODDS_1_SHUTUBA = df_config_netkeiba.loc["XPATH_ODDS_1_SHUTUBA"].iloc[0]
XPATH_ODDS_2_SHUTUBA = df_config_netkeiba.loc["XPATH_ODDS_2_SHUTUBA"].iloc[0]
XPATH_IMPOSE_1_SHUTUBA = df_config_netkeiba.loc["XPATH_IMPOSE_1_SHUTUBA"].iloc[0]
XPATH_IMPOSE_2_SHUTUBA = df_config_netkeiba.loc["XPATH_IMPOSE_2_SHUTUBA"].iloc[0]
XPATH_FRAME_1_SHUTUBA = df_config_netkeiba.loc["XPATH_FRAME_1_SHUTUBA"].iloc[0]
XPATH_FRAME_2_SHUTUBA = df_config_netkeiba.loc["XPATH_FRAME_2_SHUTUBA"].iloc[0]
XPATH_WEIGHT_1_SHUTUBA = df_config_netkeiba.loc["XPATH_WEIGHT_1_SHUTUBA"].iloc[0]
XPATH_WEIGHT_2_SHUTUBA = df_config_netkeiba.loc["XPATH_WEIGHT_2_SHUTUBA"].iloc[0]
XPATH_SEX_AND_AGE_1_SHUTUBA = df_config_netkeiba.loc["XPATH_SEX_AND_AGE_1_SHUTUBA"].iloc[0]
XPATH_SEX_AND_AGE_2_SHUTUBA = df_config_netkeiba.loc["XPATH_SEX_AND_AGE_2_SHUTUBA"].iloc[0]
XPATH_LAST_3_FURLONGS_PRED_1_SHUTUBA = df_config_netkeiba.loc["XPATH_LAST_3_FURLONGS_PRED_1_SHUTUBA"].iloc[0]
XPATH_LAST_3_FURLONGS_PRED_2_SHUTUBA = df_config_netkeiba.loc["XPATH_LAST_3_FURLONGS_PRED_2_SHUTUBA"].iloc[0]
# 馬単位（result）
XPATH_FINISH_RANK_1_RESULT = df_config_netkeiba.loc["XPATH_FINISH_RANK_1_RESULT"].iloc[0]
XPATH_FINISH_RANK_2_RESULT = df_config_netkeiba.loc["XPATH_FINISH_RANK_2_RESULT"].iloc[0]
XPATH_FRAME_NUMBER_1_RESULT = df_config_netkeiba.loc["XPATH_FRAME_NUMBER_1_RESULT"].iloc[0]
XPATH_FRAME_NUMBER_2_RESULT = df_config_netkeiba.loc["XPATH_FRAME_NUMBER_2_RESULT"].iloc[0]
XPATH_HORSE_NUMBER_1_RESULT = df_config_netkeiba.loc["XPATH_HORSE_NUMBER_1_RESULT"].iloc[0]
XPATH_HORSE_NUMBER_2_RESULT = df_config_netkeiba.loc["XPATH_HORSE_NUMBER_2_RESULT"].iloc[0]
XPATH_HORSE_LINK_1_RESULT = df_config_netkeiba.loc["XPATH_HORSE_LINK_1_RESULT"].iloc[0]
XPATH_HORSE_LINK_2_RESULT = df_config_netkeiba.loc["XPATH_HORSE_LINK_2_RESULT"].iloc[0]
XPATH_SEX_AND_AGE_1_RESULT = df_config_netkeiba.loc["XPATH_SEX_AND_AGE_1_RESULT"].iloc[0]
XPATH_SEX_AND_AGE_2_RESULT = df_config_netkeiba.loc["XPATH_SEX_AND_AGE_2_RESULT"].iloc[0]
XPATH_IMPOSE_1_RESULT = df_config_netkeiba.loc["XPATH_IMPOSE_1_RESULT"].iloc[0]
XPATH_IMPOSE_2_RESULT = df_config_netkeiba.loc["XPATH_IMPOSE_2_RESULT"].iloc[0]
XPATH_JOCKEY_LINK_1_RESULT = df_config_netkeiba.loc["XPATH_JOCKEY_LINK_1_RESULT"].iloc[0]
XPATH_JOCKEY_LINK_2_RESULT = df_config_netkeiba.loc["XPATH_JOCKEY_LINK_2_RESULT"].iloc[0]
XPATH_TIME_1_RESULT = df_config_netkeiba.loc["XPATH_TIME_1_RESULT"].iloc[0]
XPATH_TIME_2_RESULT = df_config_netkeiba.loc["XPATH_TIME_2_RESULT"].iloc[0]
XPATH_DIFF_1_RESULT = df_config_netkeiba.loc["XPATH_DIFF_1_RESULT"].iloc[0]
XPATH_DIFF_2_RESULT = df_config_netkeiba.loc["XPATH_DIFF_2_RESULT"].iloc[0]
XPATH_POPULARITY_1_RESULT = df_config_netkeiba.loc["XPATH_POPULARITY_1_RESULT"].iloc[0]
XPATH_POPULARITY_2_RESULT = df_config_netkeiba.loc["XPATH_POPULARITY_2_RESULT"].iloc[0]
XPATH_ODDS_1_RESULT = df_config_netkeiba.loc["XPATH_ODDS_1_RESULT"].iloc[0]
XPATH_ODDS_2_RESULT = df_config_netkeiba.loc["XPATH_ODDS_2_RESULT"].iloc[0]
XPATH_LAST_3_FURLONGS_1_RESULT = df_config_netkeiba.loc["XPATH_LAST_3_FURLONGS_1_RESULT"].iloc[0]
XPATH_LAST_3_FURLONGS_2_RESULT = df_config_netkeiba.loc["XPATH_LAST_3_FURLONGS_2_RESULT"].iloc[0]
XPATH_WEIGHT_1_RESULT = df_config_netkeiba.loc["XPATH_WEIGHT_1_RESULT"].iloc[0]
XPATH_WEIGHT_2_RESULT = df_config_netkeiba.loc["XPATH_WEIGHT_2_RESULT"].iloc[0]

In [20]:
### クラス ###

# 馬データ（Shutuba）
@dataclass
class Horse_Shutuba:
    """出走馬のデータを保持するデータクラス"""
    horse_id: str
    horse_name: str
    jockey_id: str
    jockey_name: str
    popularity: str
    odds: str
    sex_and_age: str
    weight: str
    horse_number: str
    frame_number: str
    position_1_top_pred: str
    position_1_left_pred: str
    position_2_top_pred: str
    position_2_left_pred: str
    position_3_top_pred: str
    position_3_left_pred: str
    position_4_top_pred: str
    position_4_left_pred: str 
    position_1_top_pred_jockey_tendency: str
    position_1_left_pred_jockey_tendency: str
    position_2_top_pred_jockey_tendency: str
    position_2_left_pred_jockey_tendency: str
    position_3_top_pred_jockey_tendency: str
    position_3_left_pred_jockey_tendency: str
    position_4_top_pred_jockey_tendency: str
    position_4_left_pred_jockey_tendency: str
    style_pred: str
    impost: str
    last_3_furlongs_pred: str

# 馬データ（Result）
@dataclass
class Horse_Result:
    """出走馬のデータを保持するデータクラス"""
    finish_rank: str
    frame_number: str
    horse_number: str
    horse_id: str
    horse_name: str
    sex_and_age: str
    impost: str    
    jockey_id: str
    jockey_name: str
    time: str
    diff: str    
    popularity: str
    odds: str
    last_3_furlongs: str
    weight: str

# レースデータ（Shutuba）
T = TypeVar('T')
@dataclass
class Race:
    """レースの基本情報と出走馬リストを保持するデータクラス"""
    race_id: str
#    race_name: str
    race_date: str
    race_time: str
    num_horses: str
    race_number: int
#    weather_name: str
#    track_condition_name: str
    racecourse: str
    ground: str
    distance: str
    direction:  str
    reliability: str
    opinion: str
    rank_1_corner: str
    rank_2_corner: str
    rank_3_corner: str
    rank_4_corner: str    
    horses: List[T] = field(default_factory=list)

In [22]:
### 関数 ###

def is_xpath_present(driver, xpath):
    return len(driver.find_elements(By.XPATH, xpath)) > 0

def extract_style_values(style_str):
    """
    style文字列からtopとleftの数値を抽出する関数
    """
    # 正規表現の解説:
    # -?    : マイナス記号があってもなくても良い
    # [\d\.]+ : 数字またはドットが1回以上続く
    # ()    : このカッコ内の部分を抽出する
    top_match = re.search(r"top:\s*(-?[\d\.]+)%", style_str)
    left_match = re.search(r"left:\s*(-?[\d\.]+)%", style_str)
    # 抽出した文字列をfloatに変換
    top_val = float(top_match.group(1)) if top_match else None
    left_val = float(left_match.group(1)) if left_match else None
    # 戻り値
    return top_val, left_val

def extract_style_values(style_str):
    """
    style文字列からtopとleftの数値を抽出する関数（マイナス対応版）
    """
    # 正規表現の解説:
    # -?    : マイナス記号があってもなくても良い
    # [\d\.]+ : 数字またはドットが1回以上続く
    # ()    : このカッコ内の部分を抽出する
    top_match = re.search(r"top:\s*(-?[\d\.]+)%", style_str)
    left_match = re.search(r"left:\s*(-?[\d\.]+)%", style_str)
    
    # 抽出した文字列をfloatに変換
    top_val = float(top_match.group(1)) if top_match else None
    left_val = float(left_match.group(1)) if left_match else None
    
    return top_val, left_val

def get_position_pred(driver):
    # HorseIconクラスを持つすべての要素を取得（リスト形式で返ってくる）
    position_pred_elements = driver.find_elements(By.CLASS_NAME, "HorseIcon")
    # 結果を格納する辞書
    dic_position_pred = {}
    for element in position_pred_elements:
        horse_number = element.get_attribute("id")
        position_pred = element.get_attribute("style")
        top_val, left_val = extract_style_values(position_pred)
        dic_val = {}
        dic_val["top"] = top_val
        dic_val["left"] = left_val
        # idが空でない場合のみ追加
        if horse_number:
            dic_position_pred[horse_number] = dic_val
    # 戻り値
    return dic_position_pred

def login_netkeiba(driver):
    # ログイン
    driver.get(LOGIN_URL)
    driver.find_element(By.NAME, "login_id").send_keys(LOGIN_ID)
    pw_field = driver.find_element(By.NAME, "pswd")
    pw_field.send_keys(LOGIN_PASSWORD)
    pw_field.send_keys(Keys.ENTER)
    time.sleep(3)

def scrape_horse_shutuba_data(driver, num_horses):
    
    all_horses_data = []
    # 馬番・馬名取得
    # 馬番・馬名・馬名リンク取得
    for i in range(1, num_horses + 1):
        try:
            # 馬番
            horse_number_xpath = HORSE_NUMBER_XPATH_1_SHUTUBA + str(i) + HORSE_NUMBER_XPATH_2_SHUTUBA
            if is_xpath_present(driver, horse_number_xpath):
                horse_number = driver.find_element(By.XPATH, horse_number_xpath).text.strip()
            else:
                break
            # 馬名と馬ID
            xpath_horse_link_1 = XPATH_HORSE_LINK_1_1_SHUTUBA + str(i) + XPATH_HORSE_LINK_1_2_SHUTUBA
            xpath_horse_link_2 = XPATH_HORSE_LINK_2_1_SHUTUBA + str(i) + XPATH_HORSE_LINK_2_2_SHUTUBA
            if is_xpath_present(driver, xpath_horse_link_1):
                xpath_horse_link = xpath_horse_link_1
            elif is_xpath_present(driver, xpath_horse_link_2):
                xpath_horse_link = xpath_horse_link_2
            horse_link_elem = driver.find_element(By.XPATH, xpath_horse_link)
            horse_name = horse_link_elem.text.strip()
            horse_href = horse_link_elem.get_attribute("href")
            horse_id = horse_href.rstrip("/").split("/")[-1]
            # 騎手名と騎手ID
            xpath_jockey_link = XPATH_JOCKEY_LINK_1_SHUTUBA + str(i) + XPATH_JOCKEY_LINK_2_SHUTUBA
            jockey_link_elem = driver.find_element(By.XPATH, xpath_jockey_link)
            jockey_name = jockey_link_elem.text.strip()
            jockey_href = jockey_link_elem.get_attribute("href")
            jockey_id = jockey_href.rstrip("/").split("/")[-1]
            # 人気
            xpath_popularity = XPATH_POPULARITY_1_SHUTUBA + str(i) + XPATH_POPULARITY_2_SHUTUBA
            popularity = driver.find_element(By.XPATH, xpath_popularity).text
            # オッズ
            xpath_odds = XPATH_ODDS_1_SHUTUBA + str(i) + XPATH_ODDS_2_SHUTUBA
            odds = driver.find_element(By.XPATH, xpath_odds).text
            # 性齢
            xpath_sex_and_age = XPATH_SEX_AND_AGE_1_SHUTUBA + str(i) + XPATH_SEX_AND_AGE_2_SHUTUBA
            sex_and_age = driver.find_element(By.XPATH, xpath_sex_and_age).text
            # 馬体重
            xpath_weight = XPATH_WEIGHT_1_SHUTUBA + str(i) + XPATH_WEIGHT_2_SHUTUBA
            weight = driver.find_element(By.XPATH, xpath_weight).text
            # 枠番
            xpath_frame_number = XPATH_FRAME_1_SHUTUBA + str(i) + XPATH_FRAME_2_SHUTUBA
            frame_number = driver.find_element(By.XPATH, xpath_frame_number).text
            # 斤量
            xpath_impost = XPATH_IMPOSE_1_SHUTUBA + str(i) + XPATH_IMPOSE_2_SHUTUBA
            impost = driver.find_element(By.XPATH, xpath_impost).text
            # 後半3F
            xpath_last_3_furlongs_pred = XPATH_LAST_3_FURLONGS_PRED_1_SHUTUBA + str(i) + XPATH_LAST_3_FURLONGS_PRED_2_SHUTUBA
            if is_xpath_present(driver, xpath_last_3_furlongs_pred):
                last_3_furlongs_pred = driver.find_element(By.XPATH, xpath_last_3_furlongs_pred).text
            else:
                last_3_furlongs_pred = None
            # 馬データ作成
            horse_data = Horse_Shutuba(
                horse_id = horse_id,
                horse_name = horse_name,
                jockey_id = jockey_id,
                jockey_name = jockey_name,
                popularity = popularity,
                odds = odds,
                sex_and_age = sex_and_age,
                weight = weight,
                horse_number = horse_number,
                frame_number = frame_number,
                position_1_top_pred = None,
                position_1_left_pred = None,
                position_2_top_pred = None,
                position_2_left_pred = None,
                position_3_top_pred = None,
                position_3_left_pred = None,
                position_4_top_pred = None,
                position_4_left_pred = None,
                position_1_top_pred_jockey_tendency = None,
                position_1_left_pred_jockey_tendency = None,
                position_2_top_pred_jockey_tendency = None,
                position_2_left_pred_jockey_tendency = None,
                position_3_top_pred_jockey_tendency = None,
                position_3_left_pred_jockey_tendency = None,
                position_4_top_pred_jockey_tendency = None,
                position_4_left_pred_jockey_tendency = None,
                style_pred = None,
                impost = impost,
                last_3_furlongs_pred = last_3_furlongs_pred
            )
            
            all_horses_data.append(horse_data)
        except Exception as e:
            print(f"  馬番{i}: データ取得失敗 ({e})")
    # 戻り値
    return all_horses_data

def scrape_horse_result_data(driver, num_horses):
    
    all_horses_data = []
    # 馬番・馬名取得
    # 馬番・馬名・馬名リンク取得
    for i in range(1, num_horses + 1):
        try:
            # 着番
            xpath_finish_rank = XPATH_FINISH_RANK_1_RESULT + str(i) + XPATH_FINISH_RANK_2_RESULT
            finish_rank = driver.find_element(By.XPATH, xpath_finish_rank).text
            # 枠番
            xpath_frame_number = XPATH_FRAME_NUMBER_1_RESULT + str(i) + XPATH_FRAME_NUMBER_2_RESULT
            frame_number = driver.find_element(By.XPATH, xpath_frame_number).text
            # 馬番
            xpath_horse_number = XPATH_HORSE_NUMBER_1_RESULT + str(i) + XPATH_HORSE_NUMBER_2_RESULT
            if is_xpath_present(driver, xpath_horse_number):
                horse_number = driver.find_element(By.XPATH, xpath_horse_number).text.strip()
            else:
                break
            # 馬名と馬ID
            xpath_horse_link = XPATH_HORSE_LINK_1_RESULT + str(i) + XPATH_HORSE_LINK_2_RESULT
            horse_link_elem = driver.find_element(By.XPATH, xpath_horse_link)
            horse_name = horse_link_elem.text.strip()
            horse_href = horse_link_elem.get_attribute("href")
            horse_id = horse_href.rstrip("/").split("/")[-1]
            # 性齢
            xpath_sex_and_age = XPATH_SEX_AND_AGE_1_RESULT + str(i) + XPATH_SEX_AND_AGE_2_RESULT
            sex_and_age = driver.find_element(By.XPATH, xpath_sex_and_age).text
            # 斤量
            xpath_impost = XPATH_IMPOSE_1_RESULT + str(i) + XPATH_IMPOSE_2_RESULT
            impost = driver.find_element(By.XPATH, xpath_impost).text
            # 騎手名と騎手ID
            xpath_jockey_link = XPATH_JOCKEY_LINK_1_RESULT + str(i) + XPATH_JOCKEY_LINK_2_RESULT
            jockey_link_elem = driver.find_element(By.XPATH, xpath_jockey_link)
            jockey_name = jockey_link_elem.text.strip()
            jockey_href = jockey_link_elem.get_attribute("href")
            jockey_id = jockey_href.rstrip("/").split("/")[-1]
            # タイム
            xpath_time = XPATH_TIME_1_RESULT + str(i) + XPATH_TIME_2_RESULT
            time = driver.find_element(By.XPATH, xpath_time).text
            # 着差
            xpath_diff = XPATH_DIFF_1_RESULT + str(i) + XPATH_DIFF_2_RESULT
            diff = driver.find_element(By.XPATH, xpath_diff).text
            # 人気
            xpath_popularity = XPATH_POPULARITY_1_RESULT + str(i) + XPATH_POPULARITY_2_RESULT
            popularity = driver.find_element(By.XPATH, xpath_popularity).text
            # オッズ
            xpath_odds = XPATH_ODDS_1_RESULT + str(i) + XPATH_ODDS_2_RESULT
            odds = driver.find_element(By.XPATH, xpath_odds).text
            # 後半3F
            xpath_last_3_furlongs = XPATH_LAST_3_FURLONGS_1_RESULT + str(i) + XPATH_LAST_3_FURLONGS_2_RESULT
            last_3_furlongs = driver.find_element(By.XPATH, xpath_last_3_furlongs).text
            # 馬体重
            xpath_weight = XPATH_WEIGHT_1_RESULT + str(i) + XPATH_WEIGHT_2_RESULT
            weight = driver.find_element(By.XPATH, xpath_weight).text
            # 馬データ作成
            horse_data = Horse_Result(
                finish_rank = finish_rank,
                frame_number = frame_number,
                horse_number = horse_number,
                horse_id = horse_id,
                horse_name = horse_name,
                sex_and_age = sex_and_age,
                impost =   impost,
                jockey_id = jockey_id,
                jockey_name = jockey_name,
                time = time,
                diff = diff,
                popularity = popularity,
                odds = odds,
                last_3_furlongs = last_3_furlongs,
                weight = weight
            )
            all_horses_data.append(horse_data)
        except Exception as e:
            print(f"  馬番{i}: データ取得失敗 ({e})")
    # 戻り値
    return all_horses_data

def scrape_race_data(driver, race_id, date):

    try:
        # 競馬場
        racecourse = driver.find_element(By.CSS_SELECTOR, SELECTOR_RACECOURSE).text
        # R
        race_number = driver.find_element(By.CSS_SELECTOR, SELECTOR_RACE_NUMBER).text
        # 頭数
        num_horses = driver.find_element(By.CSS_SELECTOR, SELECTOR_NUM_HORSES).text
        num_horses = int(num_horses.replace("頭", "").strip())
        # レース情報（発走時刻、グラウンド、距離、向き）
        race_info = driver.find_element(By.XPATH, XPATH_RACE_INFO).text
        race_info = race_info.strip()
        # "/"で分割
        parts = [p.strip() for p in race_info.split("/")]            
        # 発走時刻を抽出
        race_time = parts[0].replace("発走", "").strip()
        ground_info = parts[1]
        # グラウンドを抽出
        ground_match = re.search(r"(芝|ダ)", ground_info)
        ground = ground_match.group(1) if ground_match else "不明"
        # 距離を抽出
        distance_match = re.search(r"(\d+)m", ground_info)
        distance = int(distance_match.group(1)) if distance_match else -1
        # 向きを抽出
        direction_match = re.search(r"\((左|右)\)", ground_info)
        direction = direction_match.group(1) if direction_match else "不明"
        # 馬のデータを抽出
        if MODE == "shutuba":
            horse_list = scrape_horse_shutuba_data(driver, num_horses)
        elif MODE == "result":
            horse_list = scrape_horse_result_data(driver, num_horses)
        # 競馬場が帯広でない場合は予想データを取得する
        reliability = None
        opinion = None
        rank_1_corner = None
        rank_2_corner = None
        rank_3_corner = None
        rank_4_corner = None
        if MODE == "shutuba" and racecourse != "帯広(ば)":
            # 1人気信頼度
            if is_xpath_present(driver, XPATH_RELIABILITY):
                reliability = driver.find_element(By.XPATH, XPATH_RELIABILITY).text
            # 見解
            if is_xpath_present(driver, XPATH_OPINION):
                opinion = driver.find_element(By.XPATH, XPATH_OPINION).text
            # ポジション予想（1）
            corner = driver.find_element(By.XPATH, XPATH_CORNER_1).text
            dic_position_pred = get_position_pred(driver)
            match corner:
                case "スタート後":
                    for horse in horse_list:
                        horse.position_1_top_pred = dic_position_pred[f'Horse{horse.horse_number}']["top"]
                        horse.position_1_left_pred = dic_position_pred[f'Horse{horse.horse_number}']["left"]
                case "2コーナー":
                    for horse in horse_list:
                        horse.position_2_top_pred = dic_position_pred[f'Horse{horse.horse_number}']["top"]
                        horse.position_2_left_pred = dic_position_pred[f'Horse{horse.horse_number}']["left"]
                case "3コーナー":
                    for horse in horse_list:
                        horse.position_3_top_pred = dic_position_pred[f'Horse{horse.horse_number}']["top"]
                        horse.position_3_left_pred = dic_position_pred[f'Horse{horse.horse_number}']["left"]
                case "4コーナー":
                    for horse in horse_list:
                        horse.position_4_top_pred = dic_position_pred[f'Horse{horse.horse_number}']["top"]
                        horse.position_4_left_pred = dic_position_pred[f'Horse{horse.horse_number}']["left"]
            # ポジション予想（2）
            driver.find_element(By.XPATH, XPATH_CORNER_2).click()
            corner = driver.find_element(By.XPATH, XPATH_CORNER_2).text
            dic_position_pred = get_position_pred(driver)
            # ポジション予想を馬と紐づける
            match corner:
                case "スタート後":
                    for horse in horse_list:
                        horse.position_1_top_pred = dic_position_pred[f'Horse{horse.horse_number}']["top"]
                        horse.position_1_left_pred = dic_position_pred[f'Horse{horse.horse_number}']["left"]
                case "2コーナー":
                    for horse in horse_list:
                        horse.position_2_top_pred = dic_position_pred[f'Horse{horse.horse_number}']["top"]
                        horse.position_2_left_pred = dic_position_pred[f'Horse{horse.horse_number}']["left"]
                case "3コーナー":
                    for horse in horse_list:
                        horse.position_3_top_pred = dic_position_pred[f'Horse{horse.horse_number}']["top"]
                        horse.position_3_left_pred = dic_position_pred[f'Horse{horse.horse_number}']["left"]
                case "4コーナー":
                    for horse in horse_list:
                        horse.position_4_top_pred = dic_position_pred[f'Horse{horse.horse_number}']["top"]
                        horse.position_4_left_pred = dic_position_pred[f'Horse{horse.horse_number}']["left"]
            # ポジション予想（3）
            driver.find_element(By.XPATH, XPATH_CORNER_3).click()
            corner = driver.find_element(By.XPATH, XPATH_CORNER_3).text
            dic_position_pred = get_position_pred(driver)
            # ポジション予想を馬と紐づける
            match corner:
                case "スタート後":
                    for horse in horse_list:
                        horse.position_1_top_pred = dic_position_pred[f'Horse{horse.horse_number}']["top"]
                        horse.position_1_left_pred = dic_position_pred[f'Horse{horse.horse_number}']["left"]
                case "2コーナー":
                    for horse in horse_list:
                        horse.position_2_top_pred = dic_position_pred[f'Horse{horse.horse_number}']["top"]
                        horse.position_2_left_pred = dic_position_pred[f'Horse{horse.horse_number}']["left"]
                case "3コーナー":
                    for horse in horse_list:
                        horse.position_3_top_pred = dic_position_pred[f'Horse{horse.horse_number}']["top"]
                        horse.position_3_left_pred = dic_position_pred[f'Horse{horse.horse_number}']["left"]
                case "4コーナー":
                    for horse in horse_list:
                        horse.position_4_top_pred = dic_position_pred[f'Horse{horse.horse_number}']["top"]
                        horse.position_4_left_pred = dic_position_pred[f'Horse{horse.horse_number}']["left"]
            # 「騎手傾向を考慮」をチェック
            driver.find_element(By.XPATH, "//*[@id='dev_check_01_03']").click()
            # ポジション予想（1）
            driver.find_element(By.XPATH, XPATH_CORNER_1).click()
            corner = driver.find_element(By.XPATH, XPATH_CORNER_1).text
            dic_position_pred = get_position_pred(driver)
            match corner:
                case "スタート後":
                    for horse in horse_list:
                        horse.position_1_top_pred_jockey_tendency = dic_position_pred[f'Horse{horse.horse_number}']["top"]
                        horse.position_1_left_pred_jockey_tendency = dic_position_pred[f'Horse{horse.horse_number}']["left"]
                case "2コーナー":
                    for horse in horse_list:
                        horse.position_2_top_pred_jockey_tendency = dic_position_pred[f'Horse{horse.horse_number}']["top"]
                        horse.position_2_left_pred_jockey_tendency = dic_position_pred[f'Horse{horse.horse_number}']["left"]
                case "3コーナー":
                    for horse in horse_list:
                        horse.position_3_top_pred_jockey_tendency = dic_position_pred[f'Horse{horse.horse_number}']["top"]
                        horse.position_3_left_pred_jockey_tendency = dic_position_pred[f'Horse{horse.horse_number}']["left"]
                case "4コーナー":
                    for horse in horse_list:
                        horse.position_4_top_pred_jockey_tendency = dic_position_pred[f'Horse{horse.horse_number}']["top"]
                        horse.position_4_left_pred_jockey_tendency = dic_position_pred[f'Horse{horse.horse_number}']["left"]
            # ポジション予想（2）
            driver.find_element(By.XPATH, XPATH_CORNER_2).click()
            corner = driver.find_element(By.XPATH, XPATH_CORNER_2).text
            dic_position_pred = get_position_pred(driver)
            # ポジション予想を馬と紐づける
            match corner:
                case "スタート後":
                    for horse in horse_list:
                        horse.position_1_top_pred_jockey_tendency = dic_position_pred[f'Horse{horse.horse_number}']["top"]
                        horse.position_1_left_pred_jockey_tendency = dic_position_pred[f'Horse{horse.horse_number}']["left"]
                case "2コーナー":
                    for horse in horse_list:
                        horse.position_2_top_pred_jockey_tendency = dic_position_pred[f'Horse{horse.horse_number}']["top"]
                        horse.position_2_left_pred_jockey_tendency = dic_position_pred[f'Horse{horse.horse_number}']["left"]
                case "3コーナー":
                    for horse in horse_list:
                        horse.position_3_top_pred_jockey_tendency = dic_position_pred[f'Horse{horse.horse_number}']["top"]
                        horse.position_3_left_pred_jockey_tendency = dic_position_pred[f'Horse{horse.horse_number}']["left"]
                case "4コーナー":
                    for horse in horse_list:
                        horse.position_4_top_pred_jockey_tendency = dic_position_pred[f'Horse{horse.horse_number}']["top"]
                        horse.position_4_left_pred_jockey_tendency = dic_position_pred[f'Horse{horse.horse_number}']["left"]
            # ポジション予想（3）
            driver.find_element(By.XPATH, XPATH_CORNER_3).click()
            corner = driver.find_element(By.XPATH, XPATH_CORNER_3).text
            dic_position_pred = get_position_pred(driver)
            # ポジション予想を馬と紐づける
            match corner:
                case "スタート後":
                    for horse in horse_list:
                        horse.position_1_top_pred_jockey_tendency = dic_position_pred[f'Horse{horse.horse_number}']["top"]
                        horse.position_1_left_pred_jockey_tendency = dic_position_pred[f'Horse{horse.horse_number}']["left"]
                case "2コーナー":
                    for horse in horse_list:
                        horse.position_2_top_pred_jockey_tendency = dic_position_pred[f'Horse{horse.horse_number}']["top"]
                        horse.position_2_left_pred_jockey_tendency = dic_position_pred[f'Horse{horse.horse_number}']["left"]
                case "3コーナー":
                    for horse in horse_list:
                        horse.position_3_top_pred_jockey_tendency = dic_position_pred[f'Horse{horse.horse_number}']["top"]
                        horse.position_3_left_pred_jockey_tendency = dic_position_pred[f'Horse{horse.horse_number}']["left"]
                case "4コーナー":
                    for horse in horse_list:
                        horse.position_4_top_pred_jockey_tendency = dic_position_pred[f'Horse{horse.horse_number}']["top"]
                        horse.position_4_left_pred_jockey_tendency = dic_position_pred[f'Horse{horse.horse_number}']["left"]                
            # 「旧版の展開予想に切り替える」をクリックして脚質予想を取得
            driver.find_element(By.XPATH, "//*[@id='Netkeiba_Race_Nar_Shutuba']/div[1]/div[3]/div[14]/div/div/label/span").click()
            list_lead = []
            list_stalker = []
            list_chaser = []
            list_closer = []
            # 逃げ
            for i in range(1, num_horses + 1):
                if is_xpath_present(driver, "//*[@id='Netkeiba_Race_Nar_Shutuba']/div[1]/div[3]/div[16]/table/tbody/tr[1]/td/div[" + str(i) + "]/span[1]"):
                    list_lead.append(driver.find_element(By.XPATH, "//*[@id='Netkeiba_Race_Nar_Shutuba']/div[1]/div[3]/div[16]/table/tbody/tr[1]/td/div[" + str(i) + "]/span[1]").text)
                else:
                     break
            # 先行
            for i in range(1, num_horses + 1):
                if is_xpath_present(driver, "//*[@id='Netkeiba_Race_Nar_Shutuba']/div[1]/div[3]/div[16]/table/tbody/tr[2]/td/div[" + str(i) + "]/span[1]"):
                    list_stalker.append(driver.find_element(By.XPATH, "//*[@id='Netkeiba_Race_Nar_Shutuba']/div[1]/div[3]/div[16]/table/tbody/tr[2]/td/div[" + str(i) + "]/span[1]").text)
                else:
                     break
            # 差し
            for i in range(1, num_horses + 1):
                if is_xpath_present(driver, "//*[@id='Netkeiba_Race_Nar_Shutuba']/div[1]/div[3]/div[16]/table/tbody/tr[3]/td/div[" + str(i) + "]/span[1]"):
                    list_chaser.append(driver.find_element(By.XPATH, "//*[@id='Netkeiba_Race_Nar_Shutuba']/div[1]/div[3]/div[16]/table/tbody/tr[3]/td/div[" + str(i) + "]/span[1]").text)
                else:
                     break
            # 追込
            for i in range(1, num_horses + 1):
                if is_xpath_present(driver, "//*[@id='Netkeiba_Race_Nar_Shutuba']/div[1]/div[3]/div[16]/table/tbody/tr[4]/td/div[" + str(i) + "]/span[1]"):
                    list_closer.append(driver.find_element(By.XPATH, "//*[@id='Netkeiba_Race_Nar_Shutuba']/div[1]/div[3]/div[16]/table/tbody/tr[4]/td/div[" + str(i) + "]/span[1]").text)
                else:
                     break
            # 脚質予想を馬と紐づける
            for horse in horse_list:
                if horse.horse_number in list_lead:
                    horse.style_pred = STYLE_MAP.get["1"]
                elif horse.horse_number in list_stalker:
                    horse.style_pred = STYLE_MAP.get["2"]
                elif horse.horse_number in list_chaser:
                    horse.style_pred = STYLE_MAP.get["3"]
                elif horse.horse_number in list_closer:
                    horse.style_pred = STYLE_MAP.get["4"]
        elif MODE == "result" and racecourse != "帯広(ば)":
            if is_xpath_present(driver, XPATH_RANK_1_CORNER):
                rank_1_corner = driver.find_element(By.XPATH, XPATH_RANK_1_CORNER).text
            if is_xpath_present(driver, XPATH_RANK_2_CORNER):
                rank_2_corner = driver.find_element(By.XPATH, XPATH_RANK_2_CORNER).text
            if is_xpath_present(driver, XPATH_RANK_3_CORNER):
                rank_3_corner = driver.find_element(By.XPATH, XPATH_RANK_3_CORNER).text
            if is_xpath_present(driver, XPATH_RANK_4_CORNER):
                rank_4_corner = driver.find_element(By.XPATH, XPATH_RANK_4_CORNER).text
        # レースデータに格納
        race_data = Race(
            race_id = race_id,
            # race_name
            race_date = date,
            race_time = race_time,
            num_horses = num_horses,
            race_number = race_number,
            # weather_name
            # track_condition_name,
            racecourse = racecourse,
            ground = ground,
            distance = distance,
            direction = direction,
            reliability = reliability,
            opinion = opinion,
            rank_1_corner = rank_1_corner,
            rank_2_corner = rank_2_corner,
            rank_3_corner = rank_3_corner,
            rank_4_corner = rank_4_corner,
            horses = horse_list
        )
        # 出力確認
        print(f"\n📄 年月日: {race_data.race_date} | 競馬場: {race_data.racecourse} | R: {race_data.race_number}")
        for horse in race_data.horses:
            print(f"  馬番: {horse.horse_number} | 馬名: {horse.horse_name}")
    
    except Exception as e:
        print(f"❌ Race ID: {race_id} データ取得に失敗: {e}")
        return None
    # 戻り値
    return race_data

def scrape_one_day_data(driver, date):
    """
    競馬場とRでループ
    """
    all_races_data = []
    # 競馬場のループ
    for place_code in PLACE_MAP:
        # Rのループ
        for race_num in RACE_NUMBERS:
            yyyy = date.strftime("%Y")
            mmdd = date.strftime("%m%d")
            race_id = f"{yyyy}{place_code}{mmdd}{race_num}"
            race_url = f"https://nar.netkeiba.com/race/{MODE}.html?race_id={race_id}"
            driver.get(race_url)
            time.sleep(1)
            race_data = scrape_race_data(driver, race_id, date)
            # 1Rのrace_dataがNoneだったらその競馬場では開催していないためRのループを抜けたい
            if race_data is None:
                break
            all_races_data.append(race_data)
    # 戻り値
    return all_races_data

def main_scraper(driver, date_list):
    """
    年月日でループ
    """
    all_days_data = []
    # 年月日のループ
    for date in date_list:
        # 年月日のURLでアクセス
        date_str = date.strftime("%Y%m%d")
        race_list_url = RACE_LIST_URL + date_str
        driver.get(race_list_url)
        one_day_data = scrape_one_day_data(driver, date)
        all_days_data.append(one_day_data)
    # 戻り値
    return all_days_data

def save_to_csv(final_result, filename="race_data.csv"):
    """
    スクレイピング結果（Raceオブジェクトのリスト）を平坦化してCSVに出力する
    """
    flattened_data = []

    # final_result は [ [Race1, Race2, ...], [Race1, Race2, ...] ] という2次元リスト
    for day_data in final_result:
        for race in day_data:
            # レース情報の共通項目を辞書化 (horsesリスト以外を取得)
            race_info = {
                "race_id": race.race_id,
                "race_date": race.race_date.strftime("%Y-%m-%d") if isinstance(race.race_date, datetime) else race.race_date,
                "race_time": race.race_time,
                "num_horses": race.num_horses,
                "race_number": race.race_number,
                "racecourse": race.racecourse,
                "ground": race.ground,
                "distance": race.distance,
                "direction": race.direction,
                "reliability": race.reliability,
                "opinion": race.opinion,
                "rank_1_corner": race.rank_1_corner,
                "rank_2_corner": race.rank_2_corner,
                "rank_3_corner": race.rank_3_corner,
                "rank_4_corner": race.rank_4_corner,
            }

            # そのレースに属する馬のデータを結合
            for horse in race.horses:
                # 馬のデータを辞書化
                horse_info = vars(horse)
                
                # レース情報と馬情報を統合した新しい辞書を作成
                combined_dict = {**race_info, **horse_info}
                flattened_data.append(combined_dict)

    if not flattened_data:
        print("出力するデータがありません。")
        return

    # DataFrameに変換
    df = pd.DataFrame(flattened_data)

    # CSV出力 (日本語文字化け防止のため utf-8-sig を指定)
    df.to_csv(filename, index=False, encoding='utf-8-sig')
    print(f"✅ CSV出力が完了しました: {filename}")

In [24]:
### メイン処理 ###

# 対象期間
start_date_str = "20251129"
end_date_str = "20251129"
# スクレイピング準備
service = Service(PATH_CHROME_DRIVER)
options = webdriver.ChromeOptions()
# options.add_argument('--headless')
driver = webdriver.Chrome(service=service, options=options)
# ログイン
login_netkeiba(driver)
# 開始日～終了日
date_list = pd.date_range(
    start=start_date_str,
    end=end_date_str,
    freq='D'
)
# データ取得
final_result = main_scraper(driver, date_list)
# CSVへ保存
save_to_csv(final_result, f"C:\\Users\\ryo\\Downloads\\{MODE}_{start_date_str}-{end_date_str}.csv")
# スクレイピング終了
driver.quit()

❌ Race ID: 202535112901 データ取得に失敗: invalid literal for int() with base 10: ''
❌ Race ID: 202545112901 データ取得に失敗: invalid literal for int() with base 10: ''
❌ Race ID: 202555112901 データ取得に失敗: invalid literal for int() with base 10: ''

📄 年月日: 2025-11-29 00:00:00 | 競馬場: 帯広(ば) | R: 1R
  馬番: 4 | 馬名: トップエミー
  馬番: 10 | 馬名: センショウロック
  馬番: 7 | 馬名: ホクセイミハル
  馬番: 2 | 馬名: ホクセイゴールド
  馬番: 1 | 馬名: トカチシズ
  馬番: 8 | 馬名: マスタークィン
  馬番: 5 | 馬名: ジェイドルーリー
  馬番: 9 | 馬名: ツガルノニセイ
  馬番: 6 | 馬名: アオタイショウ
  馬番: 3 | 馬名: ジェイビアンカ

📄 年月日: 2025-11-29 00:00:00 | 競馬場: 帯広(ば) | R: 2R
  馬番: 6 | 馬名: トヨミヒラリ
  馬番: 2 | 馬名: フジベンケイ
  馬番: 8 | 馬名: チェルシーヴォラ
  馬番: 7 | 馬名: スターサッチャー
  馬番: 9 | 馬名: トウヤノハッピー
  馬番: 4 | 馬名: ブルーイングリーン
  馬番: 3 | 馬名: ホリセンダイヤ
  馬番: 5 | 馬名: イワキヘキレキ
  馬番: 1 | 馬名: ツガルノモライモノ
  馬番: 10 | 馬名: ヤマトオージョ

📄 年月日: 2025-11-29 00:00:00 | 競馬場: 帯広(ば) | R: 3R
  馬番: 7 | 馬名: コトミチャン
  馬番: 6 | 馬名: ホロミノリ
  馬番: 8 | 馬名: デカマル
  馬番: 5 | 馬名: ジェイノココロ
  馬番: 1 | 馬名: チコクシチャウヨ
  馬番: 2 | 馬名: ヤマトリキ
  馬番: 10 | 馬名: オオゾラダイヤ
  馬番: 9 | 馬名: ベリーグッドマン
  馬番

In [27]:
# 地方競馬 https://nar.netkeiba.com/top/race_list.html?kaisai_date=YYYYMMDD
# 波乱度 https://nar.sp.netkeiba.com/race/compatibility.html
# 調子偏差値 https://race.sp.netkeiba.com/barometer/profile.html?kaisai_date=YYYYMMDD